<a href="https://colab.research.google.com/github/vutt-ai-models/transformers_tutorials/blob/main/ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flair.models import SequenceTagger
from flair.embeddings import TransformerWordEmbeddings
from flair.data import Corpus
from flair.datasets import ColumnCorpus

In [ ]:
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '/content/drive/MyDrive/Omer_NER'
# initializing the corpus
corpus = ColumnCorpus(data_folder, columns,
                      train_file = 'train_slots.txt',
                      test_file = 'test_slots.txt',
                      dev_file = 'valid_slots.txt')

2023-06-18 05:21:09,252 Reading data from /content/drive/MyDrive/Omer_NER
2023-06-18 05:21:09,257 Train: /content/drive/MyDrive/Omer_NER/train_slots.txt
2023-06-18 05:21:09,265 Dev: /content/drive/MyDrive/Omer_NER/valid_slots.txt
2023-06-18 05:21:09,267 Test: /content/drive/MyDrive/Omer_NER/test_slots.txt


In [ ]:
# tag to predict
tag_type = 'ner'
# make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)
alist = [line.rstrip() for line in open('/content/drive/MyDrive/Omer_NER/slots.txt')]
for i in alist:
    tag_dictionary.add_item(i)

2023-06-18 05:21:11,103 Computing label dictionary. Progress:


4042it [00:00, 34930.21it/s]

2023-06-18 05:21:11,229 Dictionary created for label 'ner' with 108 values: NUMBER (seen 2097 times), MY (seen 611 times), DEVICE (seen 304 times), TOP_UP (seen 161 times), CHANGE (seen 158 times), BILL (seen 156 times), INTERNET (seen 155 times), PAYMENT (seen 140 times), BUNDLE (seen 137 times), SIM (seen 136 times), DATE (seen 126 times), CARD (seen 115 times), UPGRADING (seen 103 times), CALL (seen 99 times), NO (seen 89 times), AMOUNT (seen 88 times), ACCOUNT (seen 86 times), ACTIVATION (seen 83 times), CANCELLATION (seen 81 times), PURCHASE (seen 80 times)


In [ ]:
# 4. initialize fine-tuneable transformer embeddings WITH document context
embeddings = TransformerWordEmbeddings(model='bert-base-uncased',
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=True,
                                       use_context=True
                                       )

In [ ]:
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=tag_dictionary,
                        tag_type=tag_type,
                        use_rnn=True,
                        reproject_embeddings=True,
                        tag_format="BIO",
                        use_crf=True)

2023-06-18 05:21:18,600 SequenceTagger predicts: Dictionary with 251 tags: O, B-NUMBER, I-NUMBER, B-MY, I-MY, B-DEVICE, I-DEVICE, B-TOP_UP, I-TOP_UP, B-CHANGE, I-CHANGE, B-BILL, I-BILL, B-INTERNET, I-INTERNET, B-PAYMENT, I-PAYMENT, B-BUNDLE, I-BUNDLE, B-SIM, I-SIM, B-DATE, I-DATE, B-CARD, I-CARD, B-UPGRADING, I-UPGRADING, B-CALL, I-CALL, B-NO, I-NO, B-AMOUNT, I-AMOUNT, B-ACCOUNT, I-ACCOUNT, B-ACTIVATION, I-ACTIVATION, B-CANCELLATION, I-CANCELLATION, B-PURCHASE, I-PURCHASE, B-MESSAGES, I-MESSAGES, B-MONEY, I-MONEY, B-CONNECTION, I-CONNECTION, B-LOST, I-LOST, B-CONTRACT


In [ ]:
from flair.trainers import ModelTrainer
from flair.training_utils import AnnealOnPlateau
from torch.optim.adamw import AdamW
trainer = ModelTrainer(tagger, corpus)

trainer.train('/content/drive/MyDrive/Omer_NER',
              learning_rate=1e-4,
              mini_batch_size=32,
              max_epochs=150,
              scheduler=AnnealOnPlateau,
              optimizer=AdamW)

2023-06-18 05:21:20,065 ----------------------------------------------------------------------------------------------------
2023-06-18 05:21:20,072 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30523, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:69: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-06-18 05:21:20,105  - anneal_factor: "0.5"
2023-06-18 05:21:20,106  - max_epochs: "150"
2023-06-18 05:21:20,113  - shuffle: "True"
2023-06-18 05:21:20,114  - train_with_dev: "False"
2023-06-18 05:21:20,115  - batch_growth_annealing: "False"
2023-06-18 05:21:20,116 ----------------------------------------------------------------------------------------------------
2023-06-18 05:21:20,123 Model training base path: "/content/drive/MyDrive/Omer_NER"
2023-06-18 05:21:20,124 ----------------------------------------------------------------------------------------------------
2023-06-18 05:21:20,125 Device: cuda:0
2023-06-18 05:21:20,126 ----------------------------------------------------------------------------------------------------
2023-06-18 05:21:20,132 Embeddings storage mode: cpu
2023-06-18 05:21:20,133 ----------------------------------------------------------------------------------------------------
2023-06-18 05:21:33,140 epoch 1 - iter 12/127 - loss 5.83337326 - time (sec): 

100%|██████████| 24/24 [00:11<00:00,  2.15it/s]

2023-06-18 05:24:14,228 Evaluating as a multi-label problem: False
2023-06-18 05:24:14,256 DEV : loss 2.1678953170776367 - f1-score (micro avg)  0.0038
2023-06-18 05:24:14,293 BAD EPOCHS (no improvement): 0
2023-06-18 05:24:14,299 saving best model


...

2023-06-18 05:39:42,663 Evaluating as a multi-label problem: False
2023-06-18 05:39:42,689 DEV : loss 0.15095172822475433 - f1-score (micro avg)  0.8445
2023-06-18 05:39:42,731 BAD EPOCHS (no improvement): 0
2023-06-18 05:39:42,742 saving best model


2023-06-18 05:39:46,542 ----------------------------------------------------------------------------------------------------
2023-06-18 05:40:03,708 epoch 7 - iter 12/127 - loss 0.19479224 - time (sec): 17.16 - samples/sec: 259.37 - lr: 0.000100
2023-06-18 05:40:19,925 epoch 7 - iter 24/127 - loss 0.19520579 - time (sec): 33.38 - samples/sec: 258.62 - lr: 0.000100
2023-06-18 05:40:28,303 ----------------------------------------------------------------------------------------------------
2023-06-18 05:40:28,307 Exiting from training early.
2023-06-18 05:40:28,309 Saving model ...
2023-06-18 05:40:30,167 Done.
2023-06-18 05:40:30,175 ----------------------------------------------------------------------------------------------------
2023-06-18 05:40:33,273 SequenceTagger predicts: Dictionary with 253 tags: O, B-NUMBER, I-NUMBER, B-MY, I-MY, B-DEVICE, I-DEVICE, B-TOP_UP, I-TOP_UP, B-CHANGE, I-CHANGE, B-BILL, I-BILL, B-INTERNET, I-INTERNET, B-PAYMENT, I-PAYMENT, B-BUNDLE, I-BUNDLE, B-SIM, 

100%|██████████| 23/23 [00:10<00:00,  2.27it/s]

2023-06-18 05:40:43,609 Evaluating as a multi-label problem: False
2023-06-18 05:40:43,645 0.8684	0.8944	0.8812	0.8531
2023-06-18 05:40:43,647 
Results:
- F-score (micro) 0.8812
- F-score (macro) 0.481
- Accuracy 0.8531

By class:
                precision    recall  f1-score   support

        NUMBER     0.9942    1.0000    0.9971       341
            MY     0.9455    1.0000    0.9720       104
        DEVICE     0.9804    1.0000    0.9901        50
        CHANGE     1.0000    1.0000    1.0000        41
          BILL     0.9697    1.0000    0.9846        32
        TOP_UP     0.9655    1.0000    0.9825        28
      INTERNET     0.8966    1.0000    0.9455        26
       PAYMENT     0.9231    1.0000    0.9600        24
     UPGRADING     1.0000    1.0000    1.0000        25
        BUNDLE     0.6296    0.8947    0.7391        19
          CARD     0.8696    1.0000    0.9302        20
           SIM     0.9524    1.0000    0.9756        20
      CONTRACT     0.9500    1.0000    0

{'test_score': 0.8811795316565482,
 'dev_score_history': [0.0037579857196542656,
  0.39756415832970854,
  0.5504587155963302,
  0.6911083017277707,
  0.7868988391376452,
  0.8445360824742267],
 'train_loss_history': [2.1877360389245615,
  1.0588149778054796,
  0.5921017241792514,
  0.439745923691718,
  0.3281521966552171,
  0.23862481277821984],
 'dev_loss_history': [2.1678953170776367,
  0.5901839733123779,
  0.4293428957462311,
  0.29185226559638977,
  0.2131327986717224,
  0.15095172822475433]}

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load('/content/drive/MyDrive/Omer_NER/best-model.pt')
# create example sentence


2023-06-18 05:40:46,641 SequenceTagger predicts: Dictionary with 253 tags: O, B-NUMBER, I-NUMBER, B-MY, I-MY, B-DEVICE, I-DEVICE, B-TOP_UP, I-TOP_UP, B-CHANGE, I-CHANGE, B-BILL, I-BILL, B-INTERNET, I-INTERNET, B-PAYMENT, I-PAYMENT, B-BUNDLE, I-BUNDLE, B-SIM, I-SIM, B-DATE, I-DATE, B-CARD, I-CARD, B-UPGRADING, I-UPGRADING, B-CALL, I-CALL, B-NO, I-NO, B-AMOUNT, I-AMOUNT, B-ACCOUNT, I-ACCOUNT, B-ACTIVATION, I-ACTIVATION, B-CANCELLATION, I-CANCELLATION, B-PURCHASE, I-PURCHASE, B-MESSAGES, I-MESSAGES, B-MONEY, I-MONEY, B-CONNECTION, I-CONNECTION, B-LOST, I-LOST, B-CONTRACT


In [ ]:
sentence = Sentence("customer service")
# predict the tags
model.predict(sentence)


In [ ]:
print(sentence.to_tagged_string())

Sentence[2]: "customer service" → ["customer"/CONNECTION, "service"/SERVICE]
